In [252]:
import sys
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from keras.models import Sequential
from keras import layers, initializers
from keras.layers import Dense,Dropout, LSTM, Flatten
from keras.models import Model
from keras import optimizers,regularizers
from keras.callbacks import History,EarlyStopping
from keras.optimizers import SGD
import keras_metrics
from sklearn import metrics
from sklearn.utils import shuffle
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.datasets import make_classification
from sklearn.utils import shuffle, class_weight
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from keras.layers.advanced_activations import PReLU,LeakyReLU

In [253]:
labels = pd.read_csv('labels.csv',header=0)
print(list(labels.columns.values))
print("\n")
labels= labels[pd.notnull(labels['gene'])]
# df.value_counts()

    
    
data=pd.read_csv('data.csv',header=0)
print(list(data.columns.values))
data = data[pd.notnull(data['ORF'])]

['gene', 'TCA', 'Resp', 'Ribo', 'Proteas', 'Hist', 'HTH']


['ORF', 'NAME', 'alpha 0', 'alpha 7', 'alpha 14', 'alpha 21', 'alpha 28', 'alpha 35', 'alpha 42', 'alpha 49', 'alpha 56', 'alpha 63', 'alpha 70', 'alpha 77', 'alpha 84', 'alpha 91', 'alpha 98', 'alpha 105', 'alpha 112', 'alpha 119', 'Elu 0', 'Elu 30', 'Elu 60', 'Elu 90', 'Elu 120', 'Elu 150', 'Elu 180', 'Elu 210', 'Elu 240', 'Elu 270', 'Elu 300', 'Elu 330', 'Elu 360', 'Elu 390', 'cdc15 10', 'cdc15 30', 'cdc15 50', 'cdc15 70', 'cdc15 90', 'cdc15 110', 'cdc15 130', 'cdc15 150', 'cdc15 170', 'cdc15 190', 'cdc15 210', 'cdc15 230', 'cdc15 250', 'cdc15 270', 'cdc15 290', 'spo 0', 'spo 2', 'spo 5', 'spo 7', 'spo 9', 'spo 11', 'spo5 2', 'spo5 7', 'spo5 11', 'spo- early', 'spo- mid', 'heat 0', 'heat 10', 'heat 20', 'heat 40', 'heat 80', 'heat 160', 'dtt 15', 'dtt 30', 'dtt 60', 'dtt 120', 'cold 0', 'cold 20', 'cold 40', 'cold 160', 'diau a', 'diau b', 'diau c', 'diau d', 'diau e', 'diau f', 'diau g']


In [254]:
data.head(5)

,ORF,NAME,alpha 0,alpha 7,alpha 14,alpha 21,alpha 28,alpha 35,alpha 42,alpha 49,...,cold 20,cold 40,cold 160,diau a,diau b,diau c,diau d,diau e,diau f,diau g
0,YBR166C,TYR1 TYROSINE BIOSYNTHESIS PREPHENATE DEH...,0.33,-0.17,0.04,-0.07,-0.09,-0.12,-0.03,-0.20,...,0.08,0.39,-0.17,0.23,0.20,0.20,-0.17,-0.69,0.14,-0.27
1,YOR357C,GRD19 SECRETION GOLGI PROTEIN ...,-0.64,-0.38,-0.32,-0.29,-0.22,-0.01,-0.32,-0.27,...,0.36,-0.20,-0.34,-0.14,-0.09,0.06,-0.17,0.04,-0.97,-1.79
2,YLR292C,SEC72 SECRETION ER PROTEIN TRA...,-0.23,0.19,-0.36,0.14,-0.40,0.16,-0.09,-0.12,...,0.21,-0.29,-0.36,-0.10,-0.29,-0.18,-0.34,-0.47,-0.43,-1.06
3,YGL112C,TAF60 TRANSCRIPTION TFIID 60 KD SU...,-0.69,-0.89,-0.74,-0.56,-0.64,-0.18,-0.42,-0.34,...,-0.01,-0.29,0.31,0.21,0.07,0.18,-0.14,-0.20,-0.43,-1.51
4,YIL118W,RHO3 CYTOSKELETON GTP-BINDING PR...,0.04,0.01,-0.81,NaN,-0.30,0.49,0.08,0.19,...,0.28,-0.17,0.26,0.07,-0.17,NaN,-0.10,-0.23,-0.51,-1.40


In [255]:
labels.head(5)

,gene,TCA,Resp,Ribo,Proteas,Hist,HTH
1,YAL001C,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,YAL002W,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,YAL003W,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,YAL005C,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9,YAL010C,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [256]:
X=np.loadtxt('newdata.txt')
Y=np.loadtxt('newlabels.txt')
Y=Y.astype(int)

In [257]:
print(X.shape)
print(Y.shape)
X,Y= shuffle(X,Y)

(2464, 81)
(2464,)


In [258]:
n_values = np.max(Y) + 1
newY=np.eye(n_values)[Y] #one hot encoding for softmax
print(newY.shape)

(2464, 7)


In [259]:
n=int(X.shape[0]*0.80)
length=X.shape[0]
X_train=X[0:n,:]
X_test=X[n:,:]

Y_train=Y[0:n]
Y_test=Y[n:]

newY_train=newY[0:n]
newY_test=newY[n:]

In [260]:
print(X_train.shape)
print(X_test.shape)

(1971, 81)
(493, 81)


In [261]:
print(Y_train.shape)
print(Y_test.shape)

print("One hot encoded Y shape")
print(newY_train.shape)
print(newY_test.shape)

(1971,)
(493,)
One hot encoded Y shape
(1971, 7)
(493, 7)


In [262]:
#using the sklearn class weights to find find the weights of the balanced class
Y_train=Y_train.reshape((Y_train.shape[0],))
print(Y_train.shape)
class_weights_values= class_weight.compute_class_weight('balanced',np.unique(Y_train), Y_train)
Y_train=Y_train.reshape(-1, 1)

(1971,)


In [168]:
seed = 7
np.random.seed(seed)
kfold=KFold(n_splits=4, shuffle=True, random_state=seed)
accuracy=[]
f1=[]
precision=[]
recall=[]

f1_class=np.empty((0,7))
precision_class=np.empty((0,7))
recall_class=np.empty((0,7))
for train, test in kfold.split(X_train, newY_train,Y_train):
    # create model
    model = Sequential()
    model.add(Dense(175, input_dim=81, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(45, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(7, activation='softmax'))
    adam=optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['accuracy'])
    
    model.fit(X_train[train], newY_train[train], epochs=10, batch_size=10, verbose=1,class_weight=class_weights_values)
    y_pred=model.predict(X_train[test])
    predict_class = np.argmax(y_pred, axis=1)
    
    labels=[0,1,2,3,4,5,6]
    #f1 values
    f1_value=metrics.f1_score(predict_class,Y_train[test],labels,average='macro')
    f1_class_value=metrics.f1_score(predict_class,Y_train[test],labels,average=None).reshape(1, -1)
    
    #precision values
    precision_value=metrics.precision_score(predict_class,Y_train[test],labels,average='macro')
    precision_class_value=metrics.precision_score(predict_class,Y_train[test],labels,average=None).reshape(1, -1)
    
    #recall values
    recall_value=metrics.recall_score(predict_class,Y_train[test],labels,average='macro')
    recall_class_value=metrics.recall_score(predict_class,Y_train[test],labels,average=None).reshape(1, -1)
    
    scores = model.evaluate(X_train[test], newY_train[test], verbose=1)
    accuracy.append(scores[1])
    f1.append(f1_value)
    precision.append(precision_value)
    recall.append(recall_value)
    
    f1_class=np.concatenate((f1_class,f1_class_value),axis=0)
    precision_class=np.concatenate((precision_class,precision_class_value),axis=0)
    recall_class=np.concatenate((recall_class,recall_class_value),axis=0)
    
average_f1=f1_class.mean(axis=0)
average_precision=precision_class.mean(axis=0)
average_recall=recall_class.mean(axis=0)

Epoch 1/10
1478/1478 [==============================] - 11s 8ms/step - loss: 0.5056 - acc: 0.8904
Epoch 2/10
1478/1478 [==============================] - 1s 657us/step - loss: 0.2660 - acc: 0.9384
Epoch 3/10
1478/1478 [==============================] - 1s 636us/step - loss: 0.2152 - acc: 0.9526
Epoch 4/10
1478/1478 [==============================] - 1s 626us/step - loss: 0.1842 - acc: 0.9547
Epoch 5/10
1478/1478 [==============================] - 1s 596us/step - loss: 0.1397 - acc: 0.9601
Epoch 6/10
1478/1478 [==============================] - 1s 560us/step - loss: 0.1453 - acc: 0.9635
Epoch 7/10
1478/1478 [==============================] - 1s 559us/step - loss: 0.1196 - acc: 0.9682
Epoch 8/10
1478/1478 [==============================] - 1s 597us/step - loss: 0.1368 - acc: 0.9668
Epoch 9/10
1478/1478 [==============================] - 1s 552us/step - loss: 0.1142 - acc: 0.9696
Epoch 10/10
493/493 [==============================] - 4s 8ms/step
Epoch 1/10
1478/1478 [=====================

In [169]:
print("For training:")
print("Accuracy is :"+ str(np.mean(accuracy)))
print("F1 is :"+ str(np.mean(f1)))
print("Average F1 for all the classes is:" +str(average_f1))

For training:
Accuracy is :0.9710788437998501
F1 is :0.6193694770262068
Average F1 for all the classes is:[0.98418675 0.33333333 0.51515152 0.97523409 0.67351399 0.85416667
 0.        ]


In [99]:
#Based on the other models, we will compare the score against the other models
#and then see how well the model will generalise based on the test scores.

In [272]:
seed =7
np.random.seed(seed)
kfold=StratifiedKFold(n_splits=4, shuffle=True,random_state=seed)
accuracy=[]
f1=[]
precision=[]
recall=[]

f1_class=np.empty((0,7))
precision_class=np.empty((0,7))
recall_class=np.empty((0,7))

for train, test in kfold.split(X_train, Y_train):
    # create model
    
    #y_smote=np.argmax(newY_train[train], axis=1)
    
    #automatically balance all the classes of the training data
    smote_tomek = SMOTETomek(random_state=0)
    X_under, y_under=smote_tomek.fit_resample(X_train[train], Y_train[train]) #sampling on the trainng data only
    X_under,y_under= shuffle(X_under,y_under)
    
    
    n_values = np.max(y_under) + 1
    y_under=np.eye(n_values)[y_under] #one hot encoding for softmax
    act = PReLU(init='zero', weights=None)
    model = Sequential()
    model.add(Dense(175, input_dim=81, activation=PReLU())
    model.add(Dropout(0.5))
    model.add(Dense(45, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(7, activation='softmax'))
    adam=optimizers.Adam(lr=0.0025, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy',keras_metrics.precision(), keras_metrics.recall()])
    
    model.fit(X_under, y_under, epochs=10, batch_size=32, verbose=1)
    
    y_pred=model.predict(X_train[test])
    predict_class = np.argmax(y_pred, axis=1) #the predicted value
    
    labels=[0,1,2,3,4,5,6]
    Ylabel= np.argmax(newY_train[test], axis=1) #the actual value
    
    #f1 values
    f1_value=metrics.f1_score(Ylabel,predict_class,labels,average='macro')
    f1_class_value=metrics.f1_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
    
    #precision values
    precision_value=metrics.precision_score(Ylabel,predict_class,labels,average='macro')
    precision_class_value=metrics.precision_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
    
    #recall values
    recall_value=metrics.recall_score(Ylabel,predict_class,labels,average='macro')
    recall_class_value=metrics.recall_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
                                         
    scores = model.evaluate(X_train[test], newY_train[test], verbose=2)
    accuracy.append(scores[1])
    f1.append(f1_value)
    precision.append(precision_value)
    recall.append(recall_value)
    
    f1_class=np.concatenate((f1_class,f1_class_value),axis=0)
    precision_class=np.concatenate((precision_class,precision_class_value),axis=0)
    recall_class=np.concatenate((recall_class,recall_class_value),axis=0)
    
average_f1=f1_class.mean(axis=0)
average_precision=precision_class.mean(axis=0)
average_recall=recall_class.mean(axis=0)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(weights=None, alpha_initializer="zero")`
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/10
9422/9422 [==============================] - 5s 517us/step - loss: 0.3462 - acc: 0.9014 - precision: 0.9248 - recall: 0.4844
Epoch 2/10
9422/9422 [==============================] - 2s 258us/step - loss: 0.0718 - acc: 0.9804 - precision: 0.9699 - recall: 0.8863
Epoch 3/10
9422/9422 [==============================] - 2s 258us/step - loss: 0.0466 - acc: 0.9879 - precision: 0.9790 - recall: 0.9346
Epoch 4/10
9422/9422 [==============================] - 2s 260us/step - loss: 0.0347 - acc: 0.9912 - precision: 0.9892 - recall: 0.9495
Epoch 5/10
9422/9422 [==============================] - 3s 289us/step - loss: 0.0255 - acc: 0.9933 - precision: 0.9916 - recall: 0.9636
Epoch 6/10
9422/9422 [==============================] - 3s 295us/step - loss: 0.0223 - acc: 0.9940 - precision: 0.9909 - recall: 0.9673
Epoch 7/10
9422/9422 [==============================] - 3s 282us/step - loss: 0.0199 - acc: 0.9944 - precision: 0.9902 - recall: 0.9718
Epoch 8/10
9422/9422 [==========================

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `PReLU` call to the Keras 2 A

Epoch 1/10
9422/9422 [==============================] - 5s 536us/step - loss: 0.4004 - acc: 0.8807 - precision: 0.8684 - recall: 0.5000
Epoch 2/10
9422/9422 [==============================] - 3s 269us/step - loss: 0.0834 - acc: 0.9767 - precision: 0.9514 - recall: 0.8863
Epoch 3/10
9422/9422 [==============================] - 3s 309us/step - loss: 0.0417 - acc: 0.9879 - precision: 0.9776 - recall: 0.9398
Epoch 4/10
9422/9422 [==============================] - 3s 297us/step - loss: 0.0295 - acc: 0.9923 - precision: 0.9863 - recall: 0.9629
Epoch 5/10
9422/9422 [==============================] - 2s 260us/step - loss: 0.0264 - acc: 0.9923 - precision: 0.9878 - recall: 0.9591
Epoch 6/10
9422/9422 [==============================] - 2s 242us/step - loss: 0.0202 - acc: 0.9937 - precision: 0.9924 - recall: 0.9658
Epoch 7/10
9422/9422 [==============================] - 3s 285us/step - loss: 0.0189 - acc: 0.9951 - precision: 0.9947 - recall: 0.9733
Epoch 8/10
9422/9422 [==========================

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(weights=None, alpha_initializer="zero")`
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/10
9422/9422 [==============================] - 5s 524us/step - loss: 0.3707 - acc: 0.8882 - precision: 0.8497 - recall: 0.5082
Epoch 2/10
9422/9422 [==============================] - 2s 261us/step - loss: 0.0838 - acc: 0.9760 - precision: 0.9562 - recall: 0.8759
Epoch 3/10
9422/9422 [==============================] - 3s 273us/step - loss: 0.0490 - acc: 0.9862 - precision: 0.9743 - recall: 0.9279
Epoch 4/10
9422/9422 [==============================] - 2s 259us/step - loss: 0.0307 - acc: 0.9913 - precision: 0.9877 - recall: 0.9532
Epoch 5/10
9422/9422 [==============================] - 2s 261us/step - loss: 0.0294 - acc: 0.9924 - precision: 0.9840 - recall: 0.9621
Epoch 6/10
9422/9422 [==============================] - 2s 262us/step - loss: 0.0231 - acc: 0.9943 - precision: 0.9901 - recall: 0.9688
Epoch 7/10
9422/9422 [==============================] - 2s 257us/step - loss: 0.0192 - acc: 0.9949 - precision: 0.9887 - recall: 0.9762
Epoch 8/10
9422/9422 [==========================

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(weights=None, alpha_initializer="zero")`
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/10
9429/9429 [==============================] - 5s 527us/step - loss: 0.3789 - acc: 0.8900 - precision: 0.8832 - recall: 0.5219
Epoch 2/10
9429/9429 [==============================] - 2s 260us/step - loss: 0.0729 - acc: 0.9802 - precision: 0.9624 - recall: 0.8938
Epoch 3/10
9429/9429 [==============================] - 2s 259us/step - loss: 0.0433 - acc: 0.9877 - precision: 0.9752 - recall: 0.9347
Epoch 4/10
9429/9429 [==============================] - 2s 265us/step - loss: 0.0309 - acc: 0.9918 - precision: 0.9855 - recall: 0.9569
Epoch 5/10
9429/9429 [==============================] - 2s 261us/step - loss: 0.0257 - acc: 0.9943 - precision: 0.9909 - recall: 0.9696
Epoch 6/10
9429/9429 [==============================] - 3s 267us/step - loss: 0.0237 - acc: 0.9947 - precision: 0.9924 - recall: 0.9703
Epoch 7/10
9429/9429 [==============================] - 2s 260us/step - loss: 0.0189 - acc: 0.9952 - precision: 0.9910 - recall: 0.9762
Epoch 8/10
9429/9429 [==========================

In [273]:
print("For training:")
print("Accuracy is :"+ str(np.mean(accuracy)))
print("F1 is :"+ str(np.mean(f1)))
print("Average F1 for all the classes is:" +str(average_f1))

For training:
Accuracy is :0.9665255087590969
F1 is :0.6710939526649737
Average F1 for all the classes is:[0.98206729 0.43333333 0.66193182 0.93616772 0.86987179 0.81428571
 0.        ]


In [274]:
seed = 7
np.random.seed(seed)
kfold=StratifiedKFold(n_splits=4, shuffle=True,random_state=seed)
accuracy=[]
f1=[]
precision=[]
recall=[]

f1_class=np.empty((0,7))
precision_class=np.empty((0,7))
recall_class=np.empty((0,7))
r=2

for train, test in kfold.split(X_train, Y_train):
    # create model
    
    #y_smote=np.argmax(newY_train[train], axis=1)
    
    #pca for all features
    alphaX= X_train[train][:,0+r:18+r]
    eluX= X_train[train][:,18+r:32+r]
    edcX= X_train[train][:,32+r:47+r]
    spotX= X_train[train][:,47+r:56+r]
    spoX= X_train[train][:,56+r:58+r]
    heatX= X_train[train][:,58+r:64+r]
    dttX= X_train[train][:,64+r:68+r]
    coldX= X_train[train][:,68+r:72+r]
    diauX= X_train[train][:,72+r:79+r]

    alphaX_test= X_train[test][:,0+r:18+r]
    eluX_test= X_train[test][:,18+r:32+r]
    edcX_test= X_train[test][:,32+r:47+r]
    spotX_test= X_train[test][:,47+r:56+r]
    spoX_test= X_train[test][:,56+r:58+r]
    heatX_test= X_train[test][:,58+r:64+r]
    dttX_test= X_train[test][:,64+r:68+r]
    coldX_test= X_train[test][:,68+r:72+r]
    diauX_test= X_train[test][:,72+r:79+r]
    
    pca= PCA(4)
    pcaspo = PCA(2)
    pcaed=PCA(7)
    
    alphaX= pcaed.fit_transform(alphaX)
    alphaX_test=pcaed.transform(alphaX_test)
    
    eluX= pcaed.fit_transform(eluX)
    eluX_test=pcaed.transform(eluX_test)
    
    edcX= pcaed.fit_transform(edcX)
    edcX_test=pcaed.transform(edcX_test)
    
    spotX= pca.fit_transform(spotX)
    spotX_test=pca.transform(spotX_test)
    
    spoX= pcaspo.fit_transform(spoX)
    spoX_test=pcaspo.transform(spoX_test)
    
    heatX= pcaspo.fit_transform(heatX)
    heatX_test=pcaspo.transform(heatX_test)
    
    dttX= pca.fit_transform(dttX)
    dttX_test=pca.transform(dttX_test)
    
    coldX= pcaspo.fit_transform(coldX)
    coldX_test=pcaspo.transform(coldX_test)
    
    diauX= pca.fit_transform(diauX)
    diauX_test=pca.transform(diauX_test)
    
    name_train=X_train[train][:,0:r]
    name_test=X_train[test][:,0:r]
    
    x_train=np.concatenate([name_train,alphaX,eluX,edcX,spotX,spoX,heatX,dttX,coldX,diauX],axis=1)
    x_test=np.concatenate([name_test,alphaX_test,eluX_test,edcX_test,spotX_test,spoX_test,heatX_test,dttX_test,coldX_test,diauX_test],axis=1)
    print(x_train.shape)
    print(x_test.shape)
    
    
    #over and undersampling-balancing all the classes
    smote_tomek = SMOTETomek(random_state=0)
    X_under, y_under=smote_tomek.fit_resample(x_train, Y_train[train])
    X_under,y_under= shuffle(X_under,y_under)
    
    n_values = np.max(y_under) + 1
    y_under=np.eye(n_values)[y_under] #one hot encoding for softmax
    
    """
    pca= PCA(50)
    X_under= pca.fit_transform(X_under)
    newX_test=pca.transform(X_train[test])
    """
    
    model = Sequential()
    model.add(Dense(175, input_dim=39+r, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(45))
    model.add(Dropout(0.3))
    model.add(Dense(7, activation='softmax'))
    adam=optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy',keras_metrics.precision(), keras_metrics.recall()])
    
    model.fit(X_under, y_under, epochs=10, batch_size=20, verbose=1)
    
    y_pred=model.predict(x_test)
    predict_class = np.argmax(y_pred, axis=1) #the predicted value
    
    labels=[0,1,2,3,4,5,6]
    Ylabel= np.argmax(newY_train[test], axis=1) #the actual value
    
    #f1 values
    f1_value=metrics.f1_score(Ylabel,predict_class,labels,average='macro')
    f1_class_value=metrics.f1_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
    
    #precision values
    precision_value=metrics.precision_score(Ylabel,predict_class,labels,average='macro')
    precision_class_value=metrics.precision_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
    
    #recall values
    recall_value=metrics.recall_score(Ylabel,predict_class,labels,average='macro')
    recall_class_value=metrics.recall_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
                                         
    scores = model.evaluate(x_test, newY_train[test], verbose=2)
    accuracy.append(scores[1])
    f1.append(f1_value)
    precision.append(precision_value)
    recall.append(recall_value)
    
    f1_class=np.concatenate((f1_class,f1_class_value),axis=0)
    precision_class=np.concatenate((precision_class,precision_class_value),axis=0)
    recall_class=np.concatenate((recall_class,recall_class_value),axis=0)
    
average_f1=f1_class.mean(axis=0)
average_precision=precision_class.mean(axis=0)
average_recall=recall_class.mean(axis=0)

(1475, 41)
(496, 41)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/10
9422/9422 [==============================] - 6s 678us/step - loss: 0.2529 - acc: 0.9246 - precision: 0.9382 - recall: 0.6315
Epoch 2/10
9422/9422 [==============================] - 4s 408us/step - loss: 0.0506 - acc: 0.9854 - precision: 0.9718 - recall: 0.9220
Epoch 3/10
9422/9422 [==============================] - 4s 409us/step - loss: 0.0290 - acc: 0.9924 - precision: 0.9900 - recall: 0.9554
Epoch 4/10
9422/9422 [==============================] - 4s 411us/step - loss: 0.0242 - acc: 0.9936 - precision: 0.9901 - recall: 0.9666
Epoch 5/10
9422/9422 [==============================] - 5s 482us/step - loss: 0.0213 - acc: 0.9951 - precision: 0.9939 - recall: 0.9718
Epoch 6/10
9422/9422 [==============================] - 4s 434us/step - loss: 0.0205 - acc: 0.9949 - precision: 0.9902 - recall: 0.9740
Epoch 7/10
9422/9422 [==============================] - 5s 481us/step - loss: 0.0182 - acc: 0.9951 - precision: 0.9924 - recall: 0.9747
Epoch 8/10
9422/9422 [==========================

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/10
9422/9422 [==============================] - 7s 753us/step - loss: 0.2753 - acc: 0.9167 - precision: 0.9405 - recall: 0.6226
Epoch 2/10
9422/9422 [==============================] - 4s 449us/step - loss: 0.0540 - acc: 0.9851 - precision: 0.9787 - recall: 0.9212
Epoch 3/10
9422/9422 [==============================] - 4s 474us/step - loss: 0.0323 - acc: 0.9906 - precision: 0.9809 - recall: 0.9547
Epoch 4/10
9422/9422 [==============================] - 5s 495us/step - loss: 0.0186 - acc: 0.9949 - precision: 0.9932 - recall: 0.9695
Epoch 5/10
9422/9422 [==============================] - 5s 484us/step - loss: 0.0237 - acc: 0.9949 - precision: 0.9939 - recall: 0.9710
Epoch 6/10
9422/9422 [==============================] - 4s 439us/step - loss: 0.0186 - acc: 0.9956 - precision: 0.9977 - recall: 0.9725
Epoch 7/10
9422/9422 [==============================] - 4s 451us/step - loss: 0.0139 - acc: 0.9965 - precision: 0.9970 - recall: 0.9792
Epoch 8/10
9422/9422 [==========================

...

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(1482, 41)
(489, 41)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/10
9429/9429 [==============================] - 8s 799us/step - loss: 0.2711 - acc: 0.9169 - precision: 0.9226 - recall: 0.6021
Epoch 2/10
9429/9429 [==============================] - 5s 498us/step - loss: 0.0551 - acc: 0.9850 - precision: 0.9741 - recall: 0.9198
Epoch 3/10
9429/9429 [==============================] - 5s 512us/step - loss: 0.0312 - acc: 0.9912 - precision: 0.9862 - recall: 0.9525
Epoch 4/10
9429/9429 [==============================] - 5s 529us/step - loss: 0.0262 - acc: 0.9933 - precision: 0.9879 - recall: 0.9659
Epoch 5/10
9429/9429 [==============================] - 5s 498us/step - loss: 0.0211 - acc: 0.9948 - precision: 0.9954 - recall: 0.9673
Epoch 6/10
9429/9429 [==============================] - 4s 453us/step - loss: 0.0192 - acc: 0.9952 - precision: 0.9917 - recall: 0.9740
Epoch 7/10
9429/9429 [==============================] - 4s 422us/step - loss: 0.0163 - acc: 0.9954 - precision: 0.9924 - recall: 0.9755
Epoch 8/10
9429/9429 [==========================

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [275]:
print("For training:")
print("Accuracy is :"+ str(np.mean(accuracy)))
print("F1 is :"+ str(np.mean(f1)))
print("Average F1 for all the classes is:" +str(average_f1))

For training:
Accuracy is :0.973636439070516
F1 is :0.6853264992876749
Average F1 for all the classes is:[0.98575721 0.35416667 0.76162587 0.94963185 0.8461039  0.9
 0.        ]
